# Jupyter notebook to pull Sentinel-1 data from Google Earth Engine

Created by: Brandi Downs <br>
brandi.downs@jpl.nasa.gov

In [27]:
# Import libraries

import ee
import geemap
from datetime import datetime, timedelta
import math
import matplotlib.pyplot as plt
import numpy as np
import ipynb
import time
from speckle_filter import leesigma

In [29]:
# Get authentication credentials to initialize ee

ee.Authenticate()
ee.Initialize()

In [30]:
# Define geometry for data collection

# Define rectangle bounding box:
# Syntax: ee.Geometry.Rectangle([lower_left_lon, lower_left_lat, upper_right_lon, upper_right_lat])
#roi = ee.Geometry.Rectangle([-136.54, 68.35, -132.47, 70.17]); # Mackenzie River Delta
      

# Or use shapefile to define roi boundary
shp_file = 'data/pacaya.shp'
roi = geemap.shp_to_ee(shp_file)


In [47]:
# Import s1 data from google earth engine

d1 = '2023-09-12'
d2 = '2023-09-13'

# Choose ascending or descending
#orbit_pass = 'ASCENDING'
orbit_pass = 'DESCENDING'

s1_collection = (ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT')
                 .filter(ee.Filter.eq('instrumentMode', 'IW'))
                 .filterMetadata('resolution_meters', 'equals', 10)
                 .filter(ee.Filter.eq('orbitProperties_pass', orbit_pass))
                 .filter(ee.Filter.date(d1,d2))
                 .filterBounds(roi))
print('Number of images in s1 collection:', s1_collection.size().getInfo())


# Get dates of images
acq_times = s1_collection.aggregate_array('system:time_start').getInfo()
date_list = [time.strftime('%m-%d-%Y', time.gmtime(t/1000)) for t in acq_times]
print(date_list)

Number of images in s1 collection: 2
['09-12-2023', '09-12-2023']


In [48]:
# Implement speckle filter, get vv/vh ratio, keep only bands of interest

def speckle(img):
    k = 5
    vv = img.select('VV')
    vh = img.select('VH')
    vv_vh = vv.divide(vh).rename('VV_VH')
    vv_filtered = leesigma(vv,k).rename('VV_filtered')
    vh_filtered = leesigma(vh,k).rename('VH_filtered')
    vv_vh_filtered = vv_filtered.divide(vh_filtered).rename('VV_VH_filtered')
    img = img.addBands(ee.Image([vv_vh, vv_filtered, vh_filtered, vv_vh_filtered]))
    return img

s1_collection = s1_collection.map(speckle)
s1_img = s1_collection.select(['VV_filtered','VH_filtered','VV_VH_filtered'])
type(s1_img)

ee.imagecollection.ImageCollection

In [49]:
# OPTIONAL: show data on map to make sure it looks right before exporting

# Define visibility parameters
vv_vis = {
    'min': 0,
    'max': 0.2,
    'palette': ['#000000','#ffffff']
}
vh_vis = {
    'min': 0,
    'max': 0.03,
    'palette': ['#000000','#ffffff']
}
vv_vh_vis = {
    'min': 3,
    'max': 30,
    'palette': ['#000000','#ffffff']
}


# Create map and load basemap
M = geemap.Map()
M.centerObject(roi, 8)
M.add_basemap('HYBRID')

# add s1 image to map
M.addLayer(s1_img.mean().clip(roi).select('VV_filtered'), vv_vis, 's1')

# Show roi bounding box - optional
M.addLayer(roi,{},'boundary')

# Show map
M

Map(center=[-5.176930162354333, -74.76610280207753], controls=(WidgetControl(options=['position', 'transparent…

In [50]:
# For each date, merge (mean), clip to roi, and export to Google Drive

unique_dates = np.unique(date_list)
task_id = []

for k in unique_dates:
    
    t1 = datetime.strptime(k,'%m-%d-%Y').date()
    t2 = t1 + timedelta(days=1)

    t1 = t1.strftime('%Y-%m-%d')
    t2 = t2.strftime('%Y-%m-%d')
    
    # Get s1 data
    s1_collection = (ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT')
                 .filter(ee.Filter.eq('instrumentMode', 'IW'))
                 .filterMetadata('resolution_meters', 'equals', 10)
                 .filter(ee.Filter.eq('orbitProperties_pass', orbit_pass))
                 .filter(ee.Filter.date(t1,t2))
                 .filterBounds(roi))
    s1_collection = s1_collection.map(speckle)
    s1_img = s1_collection.select(['VV_filtered','VH_filtered','VV_VH_filtered']).mean().clip(roi)
    
    # Export to drive folder
    txt = 's1_' + t1 + '_' + orbit_pass
    task = ee.batch.Export.image.toDrive(image=s1_img, description=txt, scale=100, fileNamePrefix=txt,
                                         crs='EPSG:4326', fileFormat='GeoTIFF', maxPixels=2e10, region=roi)
    task.start()
    task_id.append(task.id)    
    print(txt)    


s1_2023-09-12_DESCENDING


In [51]:
task.list()

[<Task DRYOF26UEH7XFKIDD5KP55E6 EXPORT_IMAGE: s1_2023-09-12_DESCENDING (READY)>,
 <Task DR2XWUKVISET3ZLTP7B7U63E EXPORT_IMAGE: s1_2023-09-08_ASCENDING (COMPLETED)>]